# GridET Potential ET Calculation Testing

In [46]:
from datetime import datetime, timedelta, timezone
import math
import pprint

import ee
import pandas as pd
import sqlite3

import gridet
import gridet.model as model
import gridet.nldas as nldas
import gridet.solar as solar
import gridet.utils as utils

pd.set_option('display.max_rows', None)

In [47]:
# ee.Authenticate()

In [48]:
ee.Initialize(project='ee-cmorton')

In [49]:
tgt_year = 2017

start_date = f'{tgt_year}-01-01'
end_date = f'{tgt_year+1}-01-01'

daily_eto_coll = (
    ee.ImageCollection('projects/openet/assets/reference_et/utah/gridet/daily/v0')
    .filterDate(start_date, end_date)
)
daily_meteo_coll = (
    ee.ImageCollection('projects/openet/assets/meteorology/gridet/daily')
    .filterDate(start_date, end_date)
)
# nldas_coll = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').filterDate(start_date, end_date)

initiation_coll = (
    ee.ImageCollection('projects/openet/assets/reference_et/utah/gridet/ancillary/initiation')
    .filterDate(start_date, end_date)
)
termination_coll = (
    ee.ImageCollection('projects/openet/assets/reference_et/utah/gridet/ancillary/termination')
    .filterDate(start_date, end_date)
)

mask_img = ee.Image('projects/openet/assets/reference_et/utah/gridet/ancillary/mask').toInt()
# pprint.pprint(mask_img.getInfo())

# Precomputing number of days in year for target year
days_in_year = ee.Number(ee.Date.fromYMD(tgt_year, 12, 31).getRelative('day', 'year')).int().add(1).getInfo()
print(days_in_year)


365


In [50]:
# Test point in Duchesne area for checking values
test_pnt = ee.Geometry.Point(-110.1, 40.2)
rr_params = {'reducer': ee.Reducer.first(), 'geometry': test_pnt, 'scale': 100}


In [51]:
# Check the precomputed initiaion and termination values at the test point
pprint.pprint(initiation_coll.first().reduceRegion(**rr_params).getInfo())
pprint.pprint(termination_coll.first().reduceRegion(**rr_params).getInfo())


{'alfalfa': 94, 'pasture': 87}
{'alfalfa': 288, 'pasture': 312}


In [52]:
# Helper utility functions 
def f2c(temperature):
    return (temperature - 32) * (5 / 9)

def c2f(temperature):
    return temperature * (9 / 5) + 32

# def in2mm(inches):
#     return 
    
# def days_in_year(year):
#     end_of_year = date(year, 12, 31)
#     return end_of_year.timetuple().tm_yday

def get_region_df(info):
    """Convert the output of getRegions to a pandas dataframe"""
    col_dict = {}
    info_dict = {}
    for i, k in enumerate(info[0][4:]):
        col_dict[k] = i+4
        info_dict[k] = {}
        
    for row in info[1:]:
        date = datetime.utcfromtimestamp(row[3] / 1000.0).strftime('%Y-%m-%d')
        for k, v in col_dict.items():
            info_dict[k][date] = row[col_dict[k]]
            
    return pd.DataFrame.from_dict(info_dict).sort_index()
    

In [68]:
# Compute the daily Hargreaves Reference ET and growing degree day values
# NOTE: This collection is in English units to match the thresholds in the Settings.db
def daily_summation_vars_func(img):
    # Compute Growing Degree Day and Hargreaves reference ET values for each daily meteorology image

    # Precomputed daily meteorology temperature assets are in C but need to be converted to F
    tmin = utils.ToFahrenheit(img.select(['temperature_minimum']))
    tmax = utils.ToFahrenheit(img.select(['temperature_maximum']))
    tmean = tmin.add(tmax).divide(2)

    # The doc string for the function says the solar input is extraterrestrial radiation
    #   but the GridET code in Github is using the SolarRadiation variable so using that for testing
    # TODO: double check this unit conversion
    #   The Rs values are daily sums in W m-2 d-1 (so values on the order of 10k in the summer)
    ra = img.select(['shortwave_radiation']).divide(11.63)
    # ra = img.select(['extraterrestrial_radiation']).divide(11.63)
    
    return (
        ee.Image([
            gridet.model.CalculateDailyHargreavesReferenceET(tmin, tmean, tmax, ra),
            tmean.subtract(32).max(0), 
            tmean.subtract(41).max(0), 
            tmax.clamp(50, 86).add(tmin.clamp(50, 86)).divide(2).subtract(50),
        ])
        .rename(['hargreaves', 'gdd_32', 'gdd_41', 'gdd_8650'])
        .set({'system:time_start': img.get('system:time_start')})
    )

daily_summation_vars_coll = daily_meteo_coll.map(daily_summation_vars_func)

# pprint.pprint(daily_meteo_coll.first().reduceRegion(**rr_params).getInfo())
# pprint.pprint(daily_summation_vars_coll.first().reduceRegion(**rr_params).getInfo())

# daily_summation_df = get_region_df(daily_summation_vars_coll.getRegion(test_pnt, scale=100).getInfo())
# pprint.pprint(daily_summation_df)


In [54]:
# # Testing out computing the daily Hargreaves Reference ET and growing degree day values dynamically
# # This might work for exports but is timing out during testing
# def nldas_daily_func(img):
#     hourly_images = ee.ImageCollection(ee.List(img.get('matches')))
#     # pprint.pprint(hourly_images.getInfo())
    
#     tmin = (
#         hourly_images.select(['temperature']).reduce(ee.Reducer.min())
#         #.setDefaultProjection(ee.Image(img).select([0]).projection())
#         .rename(['tmin'])
#     )
#     tmax = (
#         hourly_images.select(['temperature']).reduce(ee.Reducer.max())
#         #.setDefaultProjection(ee.Image(img).select([0]).projection())
#         .rename(['tmax'])
#     )
#     tmean = tmin.add(tmax).divide(2)

#     # CGM - The doc string for the function says the solar input is "extraterrestrial_radiation"
#     #   but the input in GridET is "SolarRadiation"
#     ra = (
#         ee.ImageCollection(ee.List(img.get('matches'))).select(['shortwave_radiation'])
#         # ee.ImageCollection(ee.List(img.get('matches'))).select(['extraterrestrial_radiation'])
#         .reduce(ee.Reducer.max())
#         .setDefaultProjection(ee.Image(img).select([0]).projection())
#         .rename(['ra'])
#     )

#     hargreaves_reference_et = gridet.model.CalculateDailyHargreavesReferenceET(tmin, tmean, tmax, ra)
    
#     return (
#         ee.Image([
#             utils.toCelsius(tmin), 
#             utils.toCelsius(tmean), 
#             utils.toCelsius(tmax), 
#             hargreaves_reference_et
#         ])
#         .rename(['tmin', 'tmean', 'tmax', 'hargreaves_reference_et'])
#         .set({'system:time_start': img.get('system:time_start')})
#     )


# daily_filter = ee.Filter.And(
#     ee.Filter.maxDifference(difference=24*60*60*1000, leftField='system:time_start', rightField='system:time_start'),
#     ee.Filter.greaterThanOrEquals(leftField='system:time_start', rightField='system:time_start')
# )
# test_daily_meteo_coll = ee.ImageCollection(
#     ee.Join.saveAll('matches')
#     .apply(
#         nldas_coll.filterMetadata('start_hour', 'equals', 0).select(['temperature']), 
#         nldas_coll.map(gridet.nldas.adjust), 
#         daily_filter)
#     .map(nldas_daily_func)
# )

# # pprint.pprint(ee.Image(test_daily_meteo_coll.first()).getInfo())
# pprint.pprint(test_daily_meteo_coll.filterDate(f'{tgt_year}-01-01', f'{tgt_year+1}-01-01').first().reduceRegion(**rr_params).getInfo())

# # nldas_hargreaves = nldas_coll.map()
# # nldas_gdd32 = 
# # nldas_gdd41 = 
# # nldas_gdd8650 = 


In [55]:
# Load the crop covers and curves data from the settings database
conn = sqlite3.connect('Settings.db')
covers_df = pd.read_sql_query("SELECT * FROM covers", conn)
curves_df = pd.read_sql_query("SELECT * FROM curves", conn)
conn.close()

covers = covers_df.set_index('CoverName').to_dict('index')
curves = curves_df.set_index('CurveName').to_dict('index')
# pprint.pprint(covers)
# pprint.pprint(curves)

cover_name = 'Alfalfa (Beef)'
#cover_name = 'Alfalfa (Dairy)'
# cover_name = 'Pasture'

curve_name = covers[cover_name]['CurveName']

pprint.pprint(covers[cover_name])
pprint.pprint(curves[curve_name])


{'CurveName': 'Alfalfa',
 'CuttingIntermediateThreshold': 27.0,
 'CuttingIntermediateType': 'Number_of_Days',
 'CuttingTerminationThreshold': 15.0,
 'CuttingTerminationType': 'Number_of_Days',
 'EffectivePrecipitation': 'USDA_1970',
 'EffectiveRootDepth': 54.0,
 'InitiationThreshold': 5.25,
 'InitiationType': 'Hargreaves_Evapotranspiration',
 'IntermediateThreshold': 760.0,
 'IntermediateType': 'Growing_Degree_Days_Base_32F',
 'KillingFrostTemperature': 28.0,
 'SpringFrostTemperature': 17.0,
 'TerminationThreshold': 280.0,
 'TerminationType': 'Growing_Degree_Days_Base_32F'}
{'BeginningCurve': 'Percent_Days',
 'BeginningValues': '0.38,0.46,0.55,0.63,0.71,0.78,0.84,0.88,0.92,0.96,1\n'
                    '0.3,0.36,0.43,0.52,0.65,0.8,0.88,0.95,0.96,0.98,1\n'
                    '0.3,0.33,0.37,0.43,0.52,0.63,0.77,0.88,0.96,0.99,1',
 'EndingCurve': 'Percent_Days',
 'EndingValues': '1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1\n'
                 '1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1\n'
                 '1,1,

In [56]:
# Compute the spring and killing frost DOY

# Assuming the cover temperatures are in F
# TODO: Long term it might make more sense to convert the cover temperatures to C 
#   when they are first read in from the database (or files) instead of in the loop
spring_frost_temp = f2c(covers[cover_name]['SpringFrostTemperature'])
killing_frost_temp = f2c(covers[cover_name]['KillingFrostTemperature'])
#print(spring_frost_temp)
#print(killing_frost_temp)

# Identify days with minimum daily temperatures below the spring and killing frost thresholds
# Select the last spring frost and first killing frost DOY
def tmin_frost_masks(img):
    img_date = ee.Date(img.get('system:time_start'))
    # img_date = ee.Image(img).date()
    img_doy = ee.Number(img_date.getRelative('day', 'year')).int().add(1)

    # Minimum daily air temperature
    tmin_img = img.select(['temperature_minimum'])
    
    # Mask is True if day is below frost threshold and in season range
    sf_mask = tmin_img.lte(spring_frost_temp).And(mask_img.add(img_doy).lt(200))
    kf_mask = tmin_img.lte(killing_frost_temp).And(mask_img.add(img_doy).gte(200))
    
    # Apply the masks to the DOY image to simplify selecting the last spring and first killing frost DOY
    sf_doy = mask_img.where(sf_mask, img_doy).rename(['spring_frost_doy'])
    kf_doy = mask_img.add(days_in_year).where(kf_mask, img_doy).rename(['killing_frost_doy'])
    
    return ee.Image([sf_doy, kf_doy]).set({'system:time_start': img.get('system:time_start')})

frost_mask_coll = ee.ImageCollection(daily_meteo_coll.map(tmin_frost_masks))

# Select the last (max) spring frost DOY and first (min) killing frost DOY
spring_frost_doy = frost_mask_coll.select(['spring_frost_doy']).max()
killing_frost_doy = frost_mask_coll.select(['killing_frost_doy']).min()

pprint.pprint(spring_frost_doy.reduceRegion(**rr_params).getInfo())
pprint.pprint(killing_frost_doy.reduceRegion(**rr_params).getInfo())


{'spring_frost_doy': 66}
{'killing_frost_doy': 288}


In [72]:
# Initiation Date Selection
print(cover_name)
print(covers[cover_name]['IntermediateType'])
print(covers[cover_name]['InitiationThreshold'])

if covers[cover_name]['InitiationType'] == 'Number_of_Day':
    # Initiation Type Day of Year
    initiation_doy = mask_img.add(covers[cover_name]['InitiationThreshold'])
else:
    # Initiation Type Variable Summation
    # Select the target band based on the summartion variable
    # TODO: Convert to a dictionary or use the type string as the band name in the daily summation collection
    if covers[cover_name]['InitiationType'] == 'Hargreaves_Evapotranspiration':
        initiation_band = 'hargreaves'
    elif covers[cover_name]['InitiationType'] == 'Growing_Degree_Days_Base_32F':
        initiation_band = 'gdd_32'
    elif covers[cover_name]['InitiationType'] == 'Growing_Degree_Days_Base_41F':
        initiation_band = 'gdd_41'
    elif covers[cover_name]['InitiationType'] == 'Growing_Degree_Days_Base_86F_and_50F':
        initiation_band = 'gdd_8650'
    else:
        ValueError(f'unexepected InitiationType value: {covers[cover_name]["InitiationType"]}')

    # TODO: Look into combining these two functions into a single operation
    def cumulate(current, previous):
        previous  = ee.List(previous);
        previousLast = previous.get(previous.length().subtract(1))
        current = ee.Image(current)
        current = current.add(ee.Image(previousLast)).copyProperties(current, ['system:time_start', 'system:index'])
        return ee.List(previous).cat(ee.List([current]))

    img_list = daily_summation_vars_coll.select([initiation_band]).toList(400, 0)
    cumulate_coll = ee.ImageCollection.fromImages(img_list.slice(1, None).iterate(cumulate, img_list.slice(0, 1)))

    def initiation_threshold_doy(img):
        date = ee.Date(img.get('system:time_start'))
        doy = ee.Number(date.getRelative('day', 'year')).int().add(1)
        return (
            mask_img.add(days_in_year)
            .where(img.gte(covers[cover_name]['InitiationThreshold']), doy)
            .set({'system:time_start': date.millis()})
        )
        
    # The minimum value will be the first DOY over the threshold
    initiation_doy = cumulate_coll.map(initiation_threshold_doy).min()

# Set Initiation Date To Last Spring Frost Date If Later 
initiation_doy = initiation_doy.max(spring_frost_doy)

initiation_doy = initiation_doy.rename('initiation_doy')
pprint.pprint(initiation_doy.reduceRegion(**rr_params).getInfo())


Alfalfa (Beef)
Growing_Degree_Days_Base_32F
5.25
{'initiation_doy': 93}


In [73]:
# Intermediate Date Selection
print(cover_name)
print(covers[cover_name]['IntermediateType'])
print(covers[cover_name]['IntermediateThreshold'])

if covers[cover_name]['IntermediateType'] == 'Number_of_Days':
    # Intermediate Type Day Of Year
    intermediate_doy = initiation_doy.add(covers[cover_name]['IntermediateThreshold'])
else:
    # Intermediate Type Variable Summation 
    # Select the target band based on the summartion variable
    # TODO: Convert to a dictionary or use the type string as the band name in the daily summation collection
    if covers[cover_name]['IntermediateType'] == 'Hargreaves_Evapotranspiration':
        intermediate_band = 'hargreaves'
    elif covers[cover_name]['IntermediateType'] == 'Growing_Degree_Days_Base_32F':
        intermediate_band = 'gdd_32'
    elif covers[cover_name]['IntermediateType'] == 'Growing_Degree_Days_Base_41F':
        intermediate_band = 'gdd_41'
    elif covers[cover_name]['IntermediateType'] == 'Growing_Degree_Days_Base_86F_and_50F':
        intermediate_band = 'gdd_8650'
    else:
        ValueError(f'unexepected IntermediateType value: {covers[cover_name]["IntermediateType"]}')

    # TODO: Look into combining with the initiation and termination cumulative sums so that it is only computed once
    def cumulate(current, previous):
        previous = ee.List(previous);
        previousLast = previous.get(previous.length().subtract(1))

        # Only accumulate values if DOY is past the Initiation DOY
        doy = ee.Number(ee.Date(ee.Image(current).get('system:time_start')).getRelative('day', 'year')).int().add(1)
        current = ee.Image(current).where(initiation_doy.gte(doy), 0)
        # current = ee.Image(current)
        
        current = current.add(ee.Image(previousLast)).copyProperties(current, ['system:time_start', 'system:index'])
        
        return ee.List(previous).cat(ee.List([current]))

    img_list = daily_summation_vars_coll.select([intermediate_band]).toList(400, 0)
    cumulate_coll = ee.ImageCollection.fromImages(img_list.slice(1, None).iterate(cumulate, img_list.slice(0, 1)))

    # cumulate_df = get_region_df(cumulate_coll.getRegion(test_pnt, scale=100).getInfo())
    # pprint.pprint(cumulate_df.head(400))
    
    def intermediate_threshold_doy(img):
        date = ee.Date(img.get('system:time_start'))
        doy = ee.Number(date.getRelative('day', 'year')).int().add(1)
        return (
            killing_frost_doy
            # mask_img.add(days_in_year)
            .where(img.gte(covers[cover_name]['IntermediateThreshold']), doy)
            .set({'system:time_start': date.millis()})
        )
        
    # The minimum value will be the first DOY over the threshold
    intermediate_doy = cumulate_coll.map(intermediate_threshold_doy).min()

# # Set Intermediate date to first killing frost date if later 
# intermediate_doy = intermediate_doy.min(killing_frost_doy)

intermediate_doy = intermediate_doy.rename('intermediate_doy')
pprint.pprint(intermediate_doy.reduceRegion(**rr_params).getInfo())


Alfalfa (Beef)
Growing_Degree_Days_Base_32F
760.0
{'intermediate_doy': 133}


In [74]:
# Termination Date Selection
print(cover_name)
print(covers[cover_name]['TerminationType'])
print(covers[cover_name]['TerminationThreshold'])

if covers[cover_name]['TerminationType'] == 'Number_of_Days':
    # Termination Type Day of Year
    termination_doy = intermediate_doy.add(covers[cover_name]['TerminationThreshold'])
else:
    # Termination Type Variable Summation 
    termination_doy = mask_img.add(days_in_year)

    # Select the target band based on the summartion variable
    # TODO: Convert to a dictionary or use the type string as the band name in the daily summation collection
    if covers[cover_name]['TerminationType'] == 'Hargreaves_Evapotranspiration':
        termination_band = 'hargreaves'
    elif covers[cover_name]['TerminationType'] == 'Growing_Degree_Days_Base_32F':
        termination_band = 'gdd_32'
    elif covers[cover_name]['TerminationType'] == 'Growing_Degree_Days_Base_41F':
        termination_band = 'gdd_41'
    elif covers[cover_name]['TerminationType'] == 'Growing_Degree_Days_Base_86F_and_50F':
        termination_band = 'gdd_8650'
    else:
        ValueError(f'unexepected TerminationType value: {covers[cover_name]["TerminationType"]}')

    def cumulate(current, previous):
        previous  = ee.List(previous);
        previousLast = previous.get(previous.length().subtract(1))

        # Only accumulate values for dates past the Intermediate DOY
        doy = ee.Number(ee.Date(ee.Image(current).get('system:time_start')).getRelative('day', 'year')).int().add(1)
        current = ee.Image(current).where(intermediate_doy.gte(doy), 0)
        # current = ee.Image(current)
        
        current = current.add(ee.Image(previousLast)).copyProperties(current, ['system:time_start', 'system:index'])
        return ee.List(previous).cat(ee.List([current]))

    img_list = daily_summation_vars_coll.select([termination_band]).toList(370, 0)
    cumulate_coll = ee.ImageCollection.fromImages(img_list.slice(1, None).iterate(cumulate, img_list.slice(0, 1)))

    # cumulate_df = get_region_df(cumulate_coll.getRegion(test_pnt, scale=100).getInfo())
    # pprint.pprint(cumulate_df.head(400))
    
    def termination_threshold_doy(img):
        date = ee.Date(img.get('system:time_start'))
        doy = ee.Number(date.getRelative('day', 'year')).int().add(1)
        return (
            killing_frost_doy
            # mask_img.add(days_in_year)
            .where(img.gte(covers[cover_name]['TerminationThreshold']), doy)
            .set({'system:time_start': date.millis()})
        )
        
    # The minimum value will be the first DOY over the threshold
    termination_doy = cumulate_coll.map(termination_threshold_doy).min()

# # Set Termination date to first killing frost date if later 
# termination_doy = termination_doy.min(killing_frost_doy)
# pprint.pprint(termination_doy.reduceRegion(**rr_params).getInfo())

termination_doy = termination_doy.rename('termination_doy')
pprint.pprint(termination_doy.reduceRegion(**rr_params).getInfo())


Alfalfa (Beef)
Growing_Degree_Days_Base_32F
280.0
{'termination_doy': 146}


In [ ]:
# # Potential Evapotranspiration Calculation
# Dim StartDate As New DateTime(Year, 1, 1)
# If StartDate < MinDate Then StartDate = MinDate
# Dim EndDate As New DateTime(Year, 12, 31)
# If EndDate > MaxDate Then EndDate = MaxDate

# Dim VariablePath As String = IO.Directory.GetFiles(IntermediateCalculationsDirectory, "*" & Cover.Variable & ".db", IO.SearchOption.AllDirectories)(0)

# For D = StartDate.DayOfYear - 1 To EndDate.DayOfYear - 1
#     Dim RecordDate = (New DateTime(Year, 1, 1)).AddDays(D)
#     Dim DayOfYear As Integer = D + 1
#     Dim CuttingCurve As Integer = 0

#     Dim ETr = Variable.ReadRaster(Year, RecordDate.Month, RecordDate.Day)
#     Dim ETp(PixelCount - 1) As Single
#     For I = 0 To PixelCount - 1
#         Dim Kc As Single = 0

#         Select Case DayOfYear
#             Case Is < Initiation(I)
#                 # No Evapotranspiration
#             Case Is <= Intermediate(I)
#                 # Initiation To Intermediate Curve Interpolation
#                 Dim EndValue = Intermediate(I)

#                 Select Case Cover.InitiationToIntermediateCurveType
#                     Case CurveType.Number_of_Days
#                         Kc = CurveDaysInterpolation(DayOfYear, Initiation(I), Cover.InitialCurve, CuttingCurve)
#                     Case CurveType.Percent_Days
#                         Kc = CurvePercentInterpolation(DayOfYear, Initiation(I), EndValue, Cover.InitialCurve, CuttingCurve)
#                 End Select
#             Case Is <= Termination(I)
#                 # Intermediate To Termination Curve Interpolation
#                 Dim EndValue = Termination(I)

#                 Select Case Cover.IntermediateToTerminationCurveType
#                     Case CurveType.Number_of_Days
#                         Kc = CurveDaysInterpolation(DayOfYear, Intermediate(I), Cover.FinalCurve, CuttingCurve)
#                     Case CurveType.Percent_Days
#                         Kc = CurvePercentInterpolation(DayOfYear, Intermediate(I), EndValue, Cover.FinalCurve, CuttingCurve)
#                 End Select

#                 If Cover.SeasonalCurveType = SeasonalCurveType.Has_Cuttings Then
#                     If DayOfYear = Termination(I) Then
#                         # Only Number_of_Days Case Included for Cutting Intermediate and Termination Thresholds
#                         Select Case CType(KillingFrost(I), Int32) - CType(Termination(I), Int32)
#                             Case Is <= 0
#                                 # End of Season
#                             Case Is > Cover.CuttingIntermediateThreshold + Cover.CuttingTerminationThreshold
#                                 CuttingCurve = 1
#                                 Initiation(I) = Termination(I) + 1
#                                 Intermediate(I) = Initiation(I) + Cover.CuttingIntermediateThreshold
#                                 Termination(I) = Intermediate(I) + Cover.CuttingTerminationThreshold
#                             Case Else
#                                 CuttingCurve = 2
#                                 Initiation(I) = Termination(I) + 1
#                                 Intermediate(I) = Initiation(I) + Cover.CuttingIntermediateThreshold
#                                 If Intermediate(I) > KillingFrost(I) Then Intermediate(I) = KillingFrost(I)
#                                 Termination(I) = Intermediate(I) + Cover.CuttingTerminationThreshold
#                                 If Termination(I) > KillingFrost(I) Then Termination(I) = KillingFrost(I)
#                         End Select
#                     End If
#                 End If
#         End Select

#         ETp(I) = Kc * ETr(I)
#     Next

#     If BackgroundWorker.CancellationPending Then
#         Exit For
#     Else
#         PotentialEvapotranspiration.WriteRaster(ETp, Year, RecordDate.Month, RecordDate.Day)
#     End If


# if Cover.SeasonalCurveType == SeasonalCurveType.Has_Cuttings:
#     PotentialEvapotranspiration.WriteDate(Termination, Year, CalculationDateType.Termination)

